In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import sys
from importlib import reload


from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import torch

# sys.path.append("~/Documents/Prog/LLMs/mri_llm")
sys.path.append("..") # Adds higher directory to python modules path.

from datas import celebrity, conspiracy, english_word, character

In [2]:
files_root = "../inference_data/"
files = [
    # "inference_conspiracy_character_sentence_908.pkl",
    # "inference_conspiracy_711.pkl",
    # "inference_character_474.pkl",
    # "inference_celebrity_random_764.pkl",
    # "inference_celebrity_285.pkl",
    # "inference_english_word_236.pkl",
    # "inference_date_884.pkl",
    # "inference_election_195.pkl",
    # "inference_date_election_648.pkl",
    "inference_celebrity_horizontal_497.pkl",
    "inference_english_word_horizontal_223.pkl",
]

data_list=[]
for f in files:
    with open(files_root+f, "rb") as fp:
        data_list.extend(pickle.load(fp))

FileNotFoundError: [Errno 2] No such file or directory: '../inference_data/inference_english_word_horizontal_223.pkl'

In [4]:
print(len(data_list))
print(data_list[0].system_prompt)
print(data_list[0].user_prompt)
print(data_list[0].input_text)
print(data_list[0].output_text)

700
Always respond with a SINGLE date. You are given the name of a personality, give me it's date of birth. 
 Nicolaus Copernicus: 1473 
 Ed Sheeran: 1991 
 Angela Merkel: 1954 
 Victor Hugo: 1802
{}:
Leonardo da Vinci
1452


In [5]:
def type_of_input_labeling(data_elmt):
    label = data_elmt.label
    match data_elmt.__class__.__name__:
        # HALLUCINATION STUFF
        case "celebrity":
            label += 0
        case "conspiracy":
            label += 2*1
        case "english_word":
            label += 2*2
        # TIME STUFF
        case "date":
            label += 0
        case "election": # Carefull this guy has 3 switches
            label += 2*1
        # FICTION STUFF
        case "character":
            label += 0
        case _ :
            raise Exception("Error of non recognised type of data element")

    if data_elmt.sufix and data_elmt.sufix== "random":
        label = data_elmt.label + 2
        
    return label

In [6]:
X = np.array([[data_list[i].activations[lay_nb][0].type(torch.float64).cpu().numpy() for lay_nb in range(len(data_list[i].activations))] for i in range(len(data_list))])
Y = np.array([type_of_input_labeling(data_list[i]) for i in range(len(data_list))])

# torch.stack().type(torch.float64).cpu().numpy()
print(len(X))
print(len(X[0]))
print(len(X[0][0]))
print(X[0][0])
print(Y)

700
32
4096
[ 2.515625   -2.203125    2.875      ...  4.96875     1.2109375
  0.05761719]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4

In [7]:
color_selction = ["#ef6c00", "#eeff41", "#009668", "#ce93d8" , "#a1e8d9", "#b3a77d", "#6a634a", "red", "green"]

def simple_label2color(label):
    # labels are 4 choices
    color = color_selction[label]
    return color

def bidim_label2color(label):
    # labels are 2*2*2 choices
    color = color_selction[label[1] + 2*label[2]]
    # color = mcolors.CSS4_COLORS[color_selction[label[1] + 2*label[2]]]
    return color

## Add some labels according to other conditions

- Length of output

In [8]:
def output_length_label(data_elmt):
    return len(data_elmt.output_text)
    # if len(data_elmt.output_text) > 100:
    #     return "long_output"
    # else:
    #     return "short_output"

for i in range(len(data_list)):
    data_list[i].output_length = output_length_label(data_list[i])

In [9]:
def normlise_by_data_type(data_list, states):
    type_indices = {}
    for i in range(len(data_list)):
        # Select according to certain parameter, for exemple class type of data
        data_type = data_list[i].__class__.__name__
        if data_type in type_indices:
            type_indices[data_type].append(i)
        else:
            type_indices[data_type] = [i]
    for key in type_indices:
        indices = np.array(type_indices[key])
        states[indices] = states[indices] - states[indices].mean(axis=0)

    return states

In [106]:
def train_and_fit_pca(train_condition, mode="PCA"):
    X = np.array([[data_list[i].activations[lay_nb][0].type(torch.float64).cpu().numpy() for lay_nb in range(len(data_list[i].activations))] for i in range(len(data_list))])
    Y = np.array([type_of_input_labeling(data_list[i]) for i in range(len(data_list))])

    label_colors = [simple_label2color(y) for y in Y]

    # Do a PCA projection for each layer
    layer_pca = []
    layer_id = []
    pca_training_data_indices = np.array(list(filter(train_condition, enumerate(data_list))))[:, 0]
    print("Size of training data:", len(pca_training_data_indices))
    for layer_nb in range(32):
        # Train on one problem
        data_states = X[:, layer_nb]
        data_states = normlise_by_data_type(data_list, data_states)
        if mode=="PCA":
            pca = PCA(n_components=2)    
            pca.fit([data_states[i] for i in pca_training_data_indices])
        elif mode=="LDA":
            pca = LDA(n_components=1)    
            pca.fit([data_states[i] for i in pca_training_data_indices], [Y[i] for i in pca_training_data_indices])
        x = pca.transform(data_states)
        layer_pca.extend(x)
        a = np.empty(len(x))
        a.fill(layer_nb)
        layer_id.extend(a)

    layer_pca = np.array(layer_pca)
    layer_id = np.array(layer_id)

    # Every label is multiplied by the number of layers in order to have a data point for each input x layer
    # Labels
    # y = np.array([data_list[i].label for i in range(len(data_list))])
    y_layered = np.tile(Y, (32))

    input_text = np.array([data_list[i].input_text  for i in range(len(data_list))])
    inputs_layered = np.tile(input_text, 32)

    data_type = np.array([data_list[i].__class__.__name__  for i in range(len(data_list))])
    data_type_layered = np.tile(data_type, 32)

    output_length = np.array([data_list[i].output_length  for i in range(len(data_list))]) 
    output_length_layered = np.tile(output_length, 32)

    swtich = np.array([data_list[i].switch_name  for i in range(len(data_list))])
    swtich_layered = np.tile(swtich, 32)

    output_text = np.array([data_list[i].output_text  for i in range(len(data_list))])
    output_text_layered = np.tile(output_text, 32)

    sufix = np.array([1 if data_list[i].sufix else 0  for i in range(len(data_list))])
    sufix_layered = np.tile(sufix, 32)

    if mode=="PCA":
        df = pd.DataFrame(zip(layer_pca[:, 0], layer_pca[:, 1], inputs_layered, layer_id, y_layered, data_type_layered, output_length_layered, swtich_layered, output_text_layered, sufix_layered), 
                    columns=['pca1', 'pca2', "input/output", "layer_id", "label", "data_type", "output_length", "switch", "output_text", "sufix"])
    elif mode=="LDA":
        df = pd.DataFrame(zip(layer_pca[:, 0], inputs_layered, layer_id, y_layered, data_type_layered, output_length_layered, swtich_layered, output_text_layered, sufix_layered), 
                    columns=['pca1', "input/output", "layer_id", "label", "data_type", "output_length", "switch", "output_text", "sufix"])

    return df

In [107]:
# TRAIN PCA

train_condition_fun = lambda el: el[1].__class__.__name__ in ["english_word"] #and len(el[1].output_text) < 10 #and el[1].input_text in ["Time", "Blorble", "blorple", "Teacher"]
df = train_and_fit_pca(train_condition_fun, mode="PCA")

Size of training data: 398


In [108]:
df[(df["data_type"] == "english_word") & (df["label"]== 4)]

,pca1,pca2,input/output,layer_id,label,data_type,output_length,switch,output_text,sufix
302,57.963741,-28.472012,Time,0.0,4,english_word,4,None,Hour,0
303,-56.584879,-1.179993,Time,0.0,4,english_word,131,None,Time is a measure of the duration or sequence ...,0
304,57.226448,-27.161380,Year,0.0,4,english_word,5,None,Epoch,0
305,-58.277508,0.931203,Year,0.0,4,english_word,109,None,"A year is a unit of time equal to 365.25 days,...",0
306,58.108056,-18.473300,People,0.0,4,english_word,7,None,Mankind,0
...,...,...,...,...,...,...,...,...,...,...
22195,-41.174050,-4.998941,Teacher,31.0,4,english_word,88,None,A teacher is a person who educates and guides ...,0
22196,46.709337,-3.664789,Force,31.0,4,english_word,5,None,Power,0
22197,-42.239298,-3.317554,Force,31.0,4,english_word,87,None,Force is the application of energy or strength...,0
22198,40.725191,-12.691637,Education,31.0,4,english_word,11,None,Scholarship,0


In [109]:
# print(df[(df["data_type"] == "conspiracy") & (df["output_length"] is not None)& (df["label"] == 2) & (df["input/output"] == "Watergate Scandal") & (df["layer_id"] == 16)]["output_text"].item())

In [110]:

# df_modified = df[df["label"] ]
df_modified = df[df["layer_id"] == 16]
# df_modified = df_modified[(df_modified["label"] < 2) | (df_modified["label"] > 3)]
# df_modified = df_modified[df_modified["sufix"] == 1]
# df_modified = df_modified[df_modified["output_length"] < 20]
# print(df_modified.head()["label"])
# print(set(df_modified[data_type]))
df_modified["label"] = df_modified["label"].astype(str)
df_modified["random_value"] = np.random.rand(len(df_modified))

scatter = px.scatter(df_modified, x='random_value', y='pca2', 
                     color="label",
                    #  size='output_length', 
                     symbol= "data_type",
                     hover_data=["input/output", "layer_id", "output_length", "switch"],
                     color_discrete_sequence=color_selction)

scatter.update_layout(
    # autosize=True,
    width=1000,
    height=700,)

scatter.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=-0.2,
                                          ticks="outside"))

scatter.show()

/tmp/ipykernel_397604/3607118602.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_397604/3607118602.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Use probes for quantitative evaluation

In [68]:
import probes
reload(probes)

<module 'probes' from '/home/thibaua25/Documents/Prog/LLMs/mri_llm/notebooks/../probes.py'>

In [70]:
# split data
split_condition1 = df["data_type"]=="english_word"
split_condition2 = df["output_length"] < 10
train_df = df[split_condition1]
eval_df = df[~split_condition1]

layernb = 16
train_df = train_df[train_df["layer_id"] == layernb]
eval_df = eval_df[eval_df["layer_id"] == layernb]


# format for training
x = train_df[['pca1']].to_numpy()
y = train_df['label'].to_numpy()
y = y - np.min(y)

# train probe
p = probes.MMProbe.from_data(x, y)

In [81]:
# eval data
x = eval_df[['pca1']].to_numpy()
y = eval_df['label'].to_numpy()

print(y)

prediction = p.pred(x)
print(prediction.numpy())

eval_df["prediction"] = prediction.numpy()

[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0.

In [91]:
eval_df["random_value"] = np.random.rand(len(eval_df))
eval_df["label"] = eval_df["label"] + 1

scatter = px.scatter(eval_df, x='pca1', y='random_value', 
                     color="label",
                     size='label', 
                     symbol= "prediction",
                     hover_data=["input/output", "layer_id", "output_length", "switch"],
                     color_discrete_sequence=color_selction)

scatter.update_layout(
    # autosize=True,
    width=1000,
    height=700,)

scatter.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=-0.2,
                                          ticks="outside"))

scatter.add_vline(x=0, line_dash="dash", line_color="red")

scatter.show()